# Precomputed Connectome Voxel-wise Map Correlations
## Author: William Drew (wdrew@bwh.harvard.edu)

This script uses the GSP1000_MF Precomputed Connectome to run a voxel-wise spatial correlation between your input map and all the voxel connectivity maps.

In [ ]:
from nilearn import image, plotting
import numpy as np
import os
import json
from nimlab import pcc_tools as pc
from nimlab import datasets as nimds
from tqdm import tqdm, trange

## Choose an ROI Path

In [ ]:
# This can be a path to a csv containing your nifti paths or a path to a single nifti
roi_path = '/PHShome/wd957/test_lesions/2mm/connectivity/sub-0005_T.nii.gz'

## Pick Map Type
- Pick `'avgr'` for AvgR maps
- Pick `'fz'` for AvgR_Fz maps
- Pick `'t'` for T Maps

In [ ]:
map_type = 't'

## Choose output directory

In [ ]:
output_dir = '/PHShome/wd957'

## Load required files

In [ ]:
# Both GSP1000_MF and yeo1000_dil precomputed connectomes are available:
#json_path = '/data/nimlab/precomputed_connectomes/Production/configs/yeo1000_dil.json'
json_path = '/data/nimlab/precomputed_connectomes/Production/configs/GSP1000_MF.json'
# Load Precomputed Connectome JSON
with open(json_path) as json_file:
    precomputed_connectome = json.load(json_file)

map_suffixes = {
    "avgr":"AvgR",
    "fz":"AvgR_Fz",
    "t":"T"
}
    
# Load in all required files
brain_mask = nimds.get_img(precomputed_connectome['mask'])
brain_masker = pc.NiftiMasker(brain_mask)
brain_size = precomputed_connectome['brain_size']
connectome_norms = image.load_img(precomputed_connectome['norm'])
connectome_stdevs = image.load_img(precomputed_connectome['std'])

chunk_dir = precomputed_connectome[map_type]
chunk_img = image.load_img(precomputed_connectome['chunk_idx'])
chunk_size = precomputed_connectome['chunk_size']
num_chunks = precomputed_connectome['num_chunks']

# Load list of ROIs
roi_extension = roi_path.split('/')[-1].split('.')[1:]

if 'csv' in roi_extension:
    with open(roi_path, newline='') as f:
        reader = csv.reader(f)
        data = list(reader)
    roi_paths = [path for line in data for path in line]
    print(f"Loaded {len(roi_paths)} ROIs...")
elif 'nii' in roi_extension:
    roi_paths = [roi_path]
else:
    raise ValueError("Input File is not a NIfTI or a CSV containing paths to a list of NIfTIs")
    
rois = {}
output_maps = {}
for roi in roi_paths:
    rois[os.path.basename(roi).split(".nii")[0]] = brain_masker.transform(roi)
    output_maps[os.path.basename(roi).split(".nii")[0]] = np.zeros(brain_size, dtype=np.float32)

## Compute voxel-wise correlations

In [ ]:
for i in trange(num_chunks):
    selected_chunk = np.equal(chunk_img.get_fdata(), i+1)
    selected_chunk_img = image.new_img_like(brain_mask, selected_chunk)
    chunk_masker = pc.NiftiMasker(selected_chunk_img)
    chunk_matrix = np.load(os.path.join(chunk_dir, f"{str(i+1)}_{map_suffixes[map_type]}.npy"))

    for roi in rois:
        correls = np.corrcoef(rois[roi], chunk_matrix)[0,1:]
        output_maps[roi] += brain_masker.transform(chunk_masker.inverse_transform(correls))

In [ ]:
for roi in output_maps:
    brain_masker.inverse_transform(output_maps[roi]).to_filename(os.path.join(output_dir, roi+"_pcc_inverse_r_map.nii.gz"))